# Assembly of *Pedicularis* PE-GBS data set

A library for 48 samples was prepared following the protocol described in Escudero et al. 2013 with the PstI restriction enzyme, followed by PCR amplification of primer ligated fragments. The library prep lacked a size selection step, which we discuss in the methods below.  The library was sequenced on one lane of an Illumina HiSeq 2000 yielding xxx reads.

### This notebook
This notebook provides a fully reproducible workflow to assemble the Yu-Eaton-Ree (2012) *Pedicularis* GBS data set. This notebook and its results files are stored in the following github repo [see git repo here](https://github.com/dereneaton/pedicularis-WB-GBS). Starting from the raw data files, we denovo assemble the data in *ipyrad* to demultiplex,  filter, and cluster reads within Samples, before clustering conensus reads between samples to identify homology, and finally filtering and formating to create output files. Analysis of the resulting files is shown in separate notebooks, again available in the [git repo](https://github.com/dereneaton/pedicularis-WB-GBS).

In [1]:
## show that this dir is a git repo (has .git file mapping to the address shown)
## this allows me to push updates to this notebook directly to github, 
## to easily share its conents with others.
! git config --get remote.origin.url

https://github.com/dereneaton/pedicularis-WB-GBS.git


### Import ipyrad and other common modules

In [15]:
## all necessary software is installed alongside ipyrad, 
## and can be installed by uncommenting the command below
# conda install -c ipyrad ipyrad -y

## import basic modules and ipyrad and print version
import os
import socket
import glob
import subprocess as sps
import numpy as np
import ipyparallel as ipp
import ipyrad as ip

print "ipyrad v.{}".format(ip.__version__)
print "ipyparallel v.{}".format(ipp.__version__)
print "numpy v.{}".format(np.__version__)

ipyrad v.0.4.9
ipyparallel v.5.2.0
numpy v.1.11.2


### The cluster
This notebook is connected to 80 cores on 5 nodes of the Farnam HPC cluster at Yale. SSH Tunneling was set up following [this tutorial](http://ipyrad.readthedocs.io/HPC_Tunnel.html). Below I use the ipyparallel Python module to show explicitly which host nodes we are connected to, and which *ipyrad* will make use of. 

In [4]:
## open direct and load-balanced views to the client
ipyclient = ipp.Client()
lbview = ipyclient.load_balanced_view()
print "{} total cores".format(len(ipyclient.ids))

## confirm we are connected to 4 8-core nodes
hosts = ipyclient[:].apply_sync(socket.gethostname)
for hostname in set(hosts):
    print("host compute node: [{} cores] on {}"\
          .format(hosts.count(hostname), hostname))

80 total cores
host compute node: [16 cores] on c13n02.farnam.hpc.yale.internal
host compute node: [16 cores] on c13n05.farnam.hpc.yale.internal
host compute node: [16 cores] on c13n04.farnam.hpc.yale.internal
host compute node: [16 cores] on c13n01.farnam.hpc.yale.internal
host compute node: [16 cores] on c13n03.farnam.hpc.yale.internal


### Set up a working directory
This notebook is run from a local directory on the HPC cluster, while the scratch dir will be used as a working directory in which all big seq files will be stored. We will transfer a few files to the local dir in the end to save them for downstream analyses and to upload to github. 

In [5]:
## create a new directory in HPC scratch dir
WORK = "/ysm-gpfs/scratch60/de243/WB-PED"
if not os.path.exists(WORK):
    os.mkdir(WORK)

## the current dir (./) in which this notebook resides
NBDIR = os.path.realpath(os.curdir)

## print both
print "working directory (WORK) = {}".format(WORK)
print "current directory (NBDIR) = {}".format(NBDIR)

working directory (WORK) = /ysm-gpfs/scratch60/de243/WB-PED
current directory (NBDIR) = /ysm-gpfs/home/de243/pedicularis-WB-GBS


### The raw data
The raw R1 and R2 data are each split into 59 gzipped files approximately 300MB in size. The barcodes file maps sample names to barcodes that are contained inline in the R1 sequences, and are 4-8bp in length. The barcodes are printed a little further below. 

In [6]:
## Locations of the raw data stored temporarily on Yale's Louise HPC cluster
## Data are also stored more permanently on local computer tinus at Yale
RAWREADS = "/ysm-gpfs/project/de243/RADSEQ_RAWS/WB-PEDICULARIS/*.fastq.gz"
BARCODES = "/ysm-gpfs/project/de243/BARCODES/WB-PED_barcodes.txt"

### Fastqc quality check

I ran the program *fastQC* on the raw data files to do a quality check, the results of which are available here [fastqc_dir](https://github.com/dereneaton/pedicularis-WB-GBS/tree/master/tmp_fastqc). Overall, quality scores were not terrible, but also not great, our biggest problem was very high adapter contamination. I will filter this out using the program *cutadapt* implemented in step2 of *ipyrad*, and discussed further below.

In [ ]:
## if needed, uncomment and run the command below to install fastqc
#conda install -c bioconda fastqc -q 

In [7]:
## create a tmp directory for fastqc outfiles (./tmp_fastqc)
QUALDIR = os.path.join(NBDIR, "fastqc")
if not os.path.exists(QUALDIR):
    os.mkdir(QUALDIR)
    
## run fastqc on all raw data files and write outputs to fastqc.
## This is parallelized by load-balancing by lbview 
jobs = {}
for infile in glob.glob(RAWREADS):
    cmd = ['fastqc', infile, '--outdir', QUALDIR, '-t', '1', '-q']
    jobs[infile] = lbview.apply_async(sps.check_output, cmd)
    
## block until all finished and print progress
ipyclient.wait_interactive()

## check for fails
for key in jobs:
    if not jobs[key].successful():
        print jobs[key].exception()

 119/119 tasks finished after 1137 s
done


### Create demultiplexed files for each Sample in *ipyrad*
We set the location to the data and barcodes info for each object, and set the max barcode mismatch parameter to zero (strict), allowing no mismatches. 

In [8]:
## create an object to demultiplex each lane
demux = ip.Assembly("WB-PED_demux")

## set basic derep parameters for the two objects
demux.set_params("project_dir", os.path.join(WORK, "demux_reads"))
demux.set_params("raw_fastq_path", RAWREADS)
demux.set_params("barcodes_path", BARCODES)
demux.set_params("max_barcode_mismatch", 0)
demux.set_params("datatype", "pairgbs")
demux.set_params("restriction_overhang", ("TGCAG", "TGCAG"))

## print params
demux.get_params()

  New Assembly: WB-PED_demux
  0   assembly_name               WB-PED_demux                                 
  1   project_dir                 /ysm-gpfs/scratch60/de243/WB-PED/demux_reads 
  2   raw_fastq_path              /ysm-gpfs/project/de243/RADSEQ_RAWS/WB-PEDICULARIS/*.fastq.gz
  3   barcodes_path               /ysm-gpfs/project/de243/BARCODES/WB-PED_barcodes.txt
  4   sorted_fastq_path                                                        
  5   assembly_method             denovo                                       
  6   reference_sequence                                                       
  7   datatype                    pairgbs                                      
  8   restriction_overhang        ('TGCAG', 'TGCAG')                           
  9   max_low_qual_bases          5                                            
  10  phred_Qscore_offset         33                                           
  11  mindepth_statistical        6                                 

In [9]:
demux.run("1")


  Assembly: WB-PED_demux
  [####################] 100%  chunking large files  | 0:00:00 | s1 | 
  [####################] 100%  sorting reads         | 0:50:52 | s1 | 
  [####################] 100%  writing/compressing   | 0:42:32 | s1 | 


### Look at demux results 
Showing just a few lines of the results below, you can see that a *ton* of reads did not match to a barcode due to Ns in the barcode region of reads for files numbered 041-050. We should allow for a mismatch in the barcode sequence to recover many of these. So we reran step 1 allowing for one bp difference. 

In [10]:
## print total
print "total reads recovered: {}\n".format(demux.stats.reads_raw.sum())

## print header, and then selected results across raw files
! head -n 1 $demux.stats_files.s1
! cat $demux.stats_files.s1 | grep 0[4-5][0-9].fastq

total reads recovered: 182369514

raw_file                               total_reads    cut_found  bar_matched
lane2_NoIndex_L002_R1_040.fastq            4000000      3998844      3288695
lane2_NoIndex_L002_R2_040.fastq            4000000      3998844      3288695
lane2_NoIndex_L002_R1_041.fastq            4000000      3999819       978178
lane2_NoIndex_L002_R2_041.fastq            4000000      3999819       978178
lane2_NoIndex_L002_R1_042.fastq            4000000      3999912       538103
lane2_NoIndex_L002_R2_042.fastq            4000000      3999912       538103
lane2_NoIndex_L002_R1_043.fastq            4000000      4000000            0
lane2_NoIndex_L002_R2_043.fastq            4000000      4000000            0
lane2_NoIndex_L002_R1_044.fastq            4000000      4000000            0
lane2_NoIndex_L002_R2_044.fastq            4000000      4000000            0
lane2_NoIndex_L002_R1_045.fastq            4000000      4000000            0
lane2_NoIndex_L002_R2_045.fastq           

In [11]:
## run with one bp mismatch
demux.set_params("max_barcode_mismatch", 1)
## the force flag tells it to overwrite the previous data
demux.run("1", force=True)


  Assembly: WB-PED_demux
    [force] overwriting fastq files previously *created by ipyrad* in:
    /ysm-gpfs/scratch60/de243/WB-PED/demux_reads/WB-PED_demux_fastqs
    This does not affect your *original/raw data files*
  [####################] 100%  chunking large files  | 0:00:00 | s1 | 
  [####################] 100%  sorting reads         | 0:52:39 | s1 | 
  [####################] 100%  writing/compressing   | 0:59:53 | s1 | 


In [12]:
## print total
print "total reads recovered: {}\n".format(demux.stats.reads_raw.sum())

## print header, and then selected results across raw files
! head -n 1 $demux.stats_files.s1
! cat $demux.stats_files.s1 | grep 0[4-5][0-9].fastq

total reads recovered: 221196301

raw_file                               total_reads    cut_found  bar_matched
lane2_NoIndex_L002_R1_040.fastq            4000000      3998844      3722871
lane2_NoIndex_L002_R2_040.fastq            4000000      3998844      3722871
lane2_NoIndex_L002_R1_041.fastq            4000000      3999819      3790942
lane2_NoIndex_L002_R2_041.fastq            4000000      3999819      3790942
lane2_NoIndex_L002_R1_042.fastq            4000000      3999912      3761225
lane2_NoIndex_L002_R2_042.fastq            4000000      3999912      3761225
lane2_NoIndex_L002_R1_043.fastq            4000000      4000000      3744178
lane2_NoIndex_L002_R2_043.fastq            4000000      4000000      3744178
lane2_NoIndex_L002_R1_044.fastq            4000000      4000000      3661280
lane2_NoIndex_L002_R2_044.fastq            4000000      4000000      3661280
lane2_NoIndex_L002_R1_045.fastq            4000000      4000000      3593439
lane2_NoIndex_L002_R2_045.fastq           

### copy the results txt file back to the local dir (git repo)
I then pushed it to the repo [(view the full results here)](https://github.com/dereneaton/pedicularis-WB-GBS/blob/master/s1_demultiplex_stats.txt)

In [13]:
## the result of this demux look better, so I copied the step1
## stats file to the local dir and pushed it to the git repo.
! cp $demux.stats_files.s1 $NBDIR

### Start an assembly of the data set
I usually prefer to start my analyses from data that are already de-multiplexed, since the demux fastq files are typically what is available to others after a study is published and that data are made available online. So here we will create an Assembly object and read in the demultiplexed data. Then we will run step 2 to filter the data and take a close look at the results. 

In [ ]:
## this will be our assembly object for steps 1-6
data = ip.Assembly("c85d5f2h5")

## (optional) set a more fine-tuned threading for our cluster
data._ipcluster["threads"] = 4

## demux data location
DEMUX = os.path.join(demux.dirs.fastqs, "*gz")

## set parameters for this assembly and print them 
data.set_params("project_dir", os.path.join(WORK, data.name))
data.set_params("sorted_fastq_path", DEMUX)
data.set_params("barcodes_path", BARCODES)
data.set_params("filter_adapters", 2)
data.set_params("datatype", "pairgbs")
data.set_params("restriction_overhang", ("TGCAG", "TGCAG"))
data.set_params("max_Hs_consens", (5, 5))
data.set_params("max_SNPs_locus", (10, 10))
data.set_params("trim_overhang", (5, 5, 5, 5))
data.get_params()

In [24]:
## run steps 1-6
data.run("12", force=True)


  Assembly: c85d5f2h5
  [####################] 100%  processing reads      | 1:56:35 | s2 | 


In [25]:
print data.stats.describe().astype(int)

       state  reads_raw  reads_passed_filter
count     48         48                   48
mean       2    4608256              3373635
std        0    2749596              2021110
min        2     778354               512561
25%        2    3209073              2230412
50%        2    4055004              3017553
75%        2    5111423              3787007
max        2   17032493             11951396


### Filtering stats
I pushed the full step 2 stats file to github [(see it here)](https://github.com/dereneaton/pedicularis-WB-GBS/blob/master/s2_rawedit_stats.txt). As you can tell from this little snippet, tons of reads were trimmed of adapters, and trimmed for quality scores.

In [26]:
## print just the first few samples
print data.stats_dfs.s2.head()

           reads_raw  trim_adapter_bp_read1  trim_adapter_bp_read2  \
L20090356    4143404                1836178                1983663   
L2DZ0984     4646226                1890089                2095516   
L2DZ0988     4948408                2195090                2371398   
L2DZ0989     4209024                1803353                1950326   
L2DZ0990     4621835                1962933                2150203   

           trim_quality_bp_read1  trim_quality_bp_read2  reads_filtered_by_Ns  \
L20090356               63255677               59275618                  1484   
L2DZ0984                74989294               69346878                  1654   
L2DZ0988                71173489               62383199                  1935   
L2DZ0989                62485246               60599462                  1577   
L2DZ0990                72114365               66128652                  1712   

           reads_filtered_by_minlen  reads_passed_filter  
L20090356                   10526

### Run remaining steps of Assembly

In [13]:
data.run("3456")


  Assembly: c85d5f2h5
  [####################] 100%  dereplicating         | 0:11:03 | s3 | 
  [####################] 100%  clustering            | 9:16:24 | s3 | 
  [####################] 100%  building clusters     | 0:02:00 | s3 | 
  [####################] 100%  chunking              | 0:00:03 | s3 | 
  [####################] 100%  aligning              | 10:42:51 | s3 | 
  [####################] 100%  concatenating         | 0:01:59 | s3 | 
  [####################] 100%  inferring [H, E]      | 3:34:46 | s4 | 
  [####################] 100%  chunking clusters     | 0:01:22 | s5 | 
  [####################] 100%  consens calling       | 0:42:29 | s5 | 
  [####################] 100%  concat/shuffle input  | 0:00:15 | s6 | 
  [####################] 100%  clustering across     | 0:28:47 | s6 | 
  [####################] 100%  building clusters     | 0:00:11 | s6 | 
  [####################] 100%  aligning clusters     | 0:12:19 | s6 | 
  [####################] 100%  database indels       

In [11]:
## quick look at which filters applied most in step 5
print data.stats_dfs.s5.describe().astype(int)

       clusters_total  filtered_by_depth  filtered_by_maxH  filtered_by_maxN  \
count              48                 48                48                48   
mean           388877              85148              1859              1063   
std            208509              48816              1335               667   
min             54200              10918               165                85   
25%            236413              50829               856               499   
50%            361790              79171              1606               968   
75%            497653             110681              2466              1442   
max           1037905             257291              6632              3083   

       reads_consens   nsites  nhetero  heterozygosity  
count             48       48       48              48  
mean           29256  3065792    39580               0  
std            16764  1831307    23804               0  
min             5323   474566     4597             

### Create final output files
We will do this for several values of `min_samples_locus`, and provide unique names for each resulting assembly. 

In [12]:
## create named branches for final assemblies
min4 = data.branch("min4_c85d5f2h5")
min4.set_params("min_samples_locus", 4)

min10 = data.branch("min10_c85d5f2h5")
min10.set_params("min_samples_locus", 10)

## assemble outfiles
min4.run("7", force=True)
min10.run("7", force=True)


  Assembly: min4_c85d5f2h5
  [####################] 100%  filtering loci        | 0:01:41 | s7 | 
  [####################] 100%  building loci/stats   | 0:00:08 | s7 | 
  [####################] 100%  building vcf file     | 0:00:51 | s7 | 
  [####################] 100%  writing vcf file      | 0:00:01 | s7 | 
  [####################] 100%  writing outfiles      | 0:00:08 | s7 | 
  Outfiles written to: /ysm-gpfs/scratch60/de243/WB-PED/c85d5f2h5/min4_c85d5f2h5_outfiles

  Assembly: min10_c85d5f2h5
  [####################] 100%  filtering loci        | 0:00:31 | s7 | 
  [####################] 100%  building loci/stats   | 0:00:08 | s7 | 
  [####################] 100%  building vcf file     | 0:00:28 | s7 | 
  [####################] 100%  writing vcf file      | 0:00:00 | s7 | 
  [####################] 100%  writing outfiles      | 0:00:01 | s7 | 
  Outfiles written to: /ysm-gpfs/scratch60/de243/WB-PED/c85d5f2h5/min10_c85d5f2h5_outfiles


### Print final stats for the min4 assembly

In [13]:
!cat $min4.stats_files.s7



## The number of loci caught by each filter.
## ipyrad API location: [assembly].statsfiles.s7_filters

                            total_filters  applied_order  retained_loci
total_prefiltered_loci             191873              0         191873
filtered_by_rm_duplicates           11199          11199         180674
filtered_by_max_indels               2155           1607         179067
filtered_by_max_snps                22951          18514         160553
filtered_by_max_shared_het             49              6         160547
filtered_by_min_sample             128346         123369          37178
filtered_by_max_alleles             60455          15075          22103
total_filtered_loci                 22103              0          22103


## The number of loci recovered for each Sample.
## ipyrad API location: [assembly].stats_dfs.s7_samples

            sample_coverage
L20090356              1869
L2DZ0984               3361
L2DZ0988               2983
L2DZ0989               2905

### A quick view of the first few loci

In [14]:
! head -n 50 $min4.outfiles.loci | cut -c 1-80

d19long1       CTACGAATTGTGTTTCGTTGGCGGACnnnnAGGAAGAAGGGTTTGACGAGGCTGAGCCATCGGTG
d30181         CTAGGTATTGTGTTTCGTTGGCGGGCnnnnAGGAAGAAGGGTTTGACGAGGCTGAGCCATCGGTG
d31733         CTAGGTATTGTGTTTCGTTGGCAGGCnnnnAGGAAGAAGGGTTTGACGAGGCTGAGCCATCGGTG
d39253         CTAGGTATTGTGTTTCGTTGGCGGACnnnnAGGAAGAAGNGTTTGACAAGGCTGAGCCATCGGTG
//                - -                - *                      -                 
d19long1       AGCATGCTNGAGTCGGGTTCGTCNTCGAGCACYTCGTTCTCNANGTCGCCGGTGCCCACCCATTG
d30695         AGCATGCTGGAGTCGGGTTCGTCGTCGAGCACYTCGTTCTCCACGTCGCCGGTGCCSACCNATTG
d41389         AGCATGCTCGAGTCGGGTTCGTCGTCGAGCACCTCGTTCTCCACGTCGCCGGTGCCGACCCATTG
decor21        AGCATGCTGGAGTCGGGTTCGTCGTCGAGCACCTCGTTCTCCACGTCGCCGGTGCCCACCCATTG
//                     -                       *                       *        
d19long1       ACTTATTCATCATCCAACGGGCGGCCTTAC
d34041         ACTTRTTCATCATCCAACGGGCGGCCTTAC
d39103         ACTTATTCATCATCCAACGGGCGGCCTTAC
d39253         ACTTATTCATCATCCAACGGGCGGCCTTAC
decor2

## Analysis methods

In [30]:
## reload, b/c I disconnected and came back.
min10 = ip.load_json("/fastscratch/de243/WB-PED/c85d5f2h5/min10_c85d5f2h5.json")
min10.outfiles.loci[:-5]+'.phy'

  loading Assembly: min4_c85d5f2h5
  from saved path: /fastscratch/de243/WB-PED/c85d5f2h5/min4_c85d5f2h5.json


'/fastscratch/de243/WB-PED/c85d5f2h5/min4_c85d5f2h5_outfiles/min4_c85d5f2h5.phy'

In [78]:
## ask if it's still running in background
if proc.poll():
    print proc.returncode
else:
    tail = ! tail $RAXDIR/RAxML_info.min10*
    print "still running. \ntail:", tail[-1]

still running. 
tail: 


### Transfer output files to the git repo for downstream analyses

In [17]:
## move data from WORK to NBDIR
print WORK
print NBDIR

! cp -r  $WORK/c85d5f2h5/min4_c85d5f2h5_outfiles/ $NBDIR
! cp -r  $WORK/c85d5f2h5/min10_c85d5f2h5_outfiles/ $NBDIR


/ysm-gpfs/scratch60/de243/WB-PED
/ysm-gpfs/home/de243/pedicularis-WB-GBS


In [6]:
import ete3 as ete

## load tree
tre = ete.Tree("analysis_raxml/RAxML_bipartitions.min4_tree", format=0)

## sub in names
for node in tre.traverse():
    if node.name in NAMES:
        node.name = NAMES[node.name]


#print tre.get_ascii(attributes=["name", "color"], show_internal=False)
tre.ladderize()
print tre.get_ascii(attributes=["name", "support"])


                   /-L2DZ1204, 1.0
            /, 100.0
           |       \-L2DZ1180, 1.0
           |
           |                   /-L2DZ1027, 1.0
           |            /, 100.0
     /, 100.0          |       \-L2DZ1006, 1.0
    |      |      /, 95.0
    |      |     |     |      /-L2DZ1268, 1.0
    |      |     |      \, 95.0
    |      |     |           |       /-L2DZ1266, 1.0
    |      |     |            \, 100.0
    |       \, 95.0                  \-L2DZ1007, 1.0
    |            |
    |            |       /-L2DZ1011, 1.0
    |            |      |
    |            |      |            /-L2DZ1070, 1.0
    |             \, 100.0     /, 97.0
    |                   |     |     |       /-LHW10346, 1.0
    |                   |     |      \, 100.0
    |                   |     |             \-L2DZ1060, 1.0
    |                    \, 83.0
    |                         |             /-L2DZ0988, 1.0
    |                         |       /, 96.0
    |                         |     

### Plot the tree in R

In [46]:
test = ip.load_json("/fastscratch/de243/WB-PED/c85d5f2h5/c85d5f2h5.json")
sample = test.samples["d19long1"]

  loading Assembly: c85d5f2h5
  from saved path: /fastscratch/de243/WB-PED/c85d5f2h5/c85d5f2h5.json
